In [1]:
from collections import deque

In [2]:
def solve_part_1(map_str, moves_str):
    # Part 1 logic: original warehouse, no scaling
    lines = map_str.strip('\n').split('\n')
    robot = (-1, -1)
    boxes = set()
    walls = set()

    deltas = {
        "^": (0, -1),
        ">": (1, 0),
        "v": (0, 1),
        "<": (-1, 0)
    }

    # Parse map
    for y, l in enumerate(lines):
        for x, h in enumerate(l):
            if h == "O":
                boxes.add((x, y))
            elif h == "#":
                walls.add((x, y))
            elif h == "@":
                robot = (x, y)

    moves = moves_str.replace("\n", "")

    def free_space(robot, d, walls, boxes):
        dx, dy = d
        cx, cy = robot[0] + dx, robot[1] + dy
        while True:
            if (cx, cy) in walls:
                return False, (-1, -1)
            if (cx, cy) in boxes:
                cx, cy = cx + dx, cy + dy
            else:
                return True, (cx, cy)

    # Simulate moves
    for m in moves:
        dx, dy = deltas[m]
        nx, ny = robot[0] + dx, robot[1] + dy
        if (nx, ny) not in walls:
            if (nx, ny) not in boxes:
                # Just move robot
                robot = (nx, ny)
            else:
                # Attempt to push boxes
                is_free, next_free = free_space(robot, (dx, dy), walls, boxes)
                if is_free:
                    to_remove = set()
                    to_add = set()
                    # Move all boxes in line
                    xs = [robot[0], next_free[0]]
                    ys = [robot[1], next_free[1]]
                    min_x, max_x = min(xs), max(xs)
                    min_y, max_y = min(ys), max(ys)
                    for b in boxes:
                        if min_x <= b[0] <= max_x and min_y <= b[1] <= max_y:
                            to_remove.add(b)
                            to_add.add((b[0] + dx, b[1] + dy))
                    boxes -= to_remove
                    boxes |= to_add
                    robot = (robot[0]+dx, robot[1]+dy)

    # Compute sum of GPS coordinates
    return sum(b[0] + 100*b[1] for b in boxes)

In [3]:
def solve_part_2(map_str, moves_str):
    # Part 2 logic: scaled warehouse
    # Scaling rules:
    # '#' -> '##'
    # 'O' -> '[]'
    # '.' -> '..'
    # '@' -> '@.'

    lines = map_str.strip('\n').split('\n')
    rows = len(lines)
    cols = len(lines[0])

    scaled_lines = []
    for l in lines:
        new_line = []
        for ch in l:
            if ch == '#':
                new_line.append("##")
            elif ch == 'O':
                new_line.append("[]")
            elif ch == '.':
                new_line.append("..")
            elif ch == '@':
                new_line.append("@.")
            else:
                new_line.append("..")
        scaled_lines.append("".join(new_line))

    # Convert to coordinates:
    # Now, walls and boxes occupy pairs of horizontally adjacent cells.
    # Robot occupies '@.' at some position.
    R = rows
    C = cols * 2

    walls = set()
    boxes = set()
    robot = (-1, -1)

    for y in range(R):
        line = scaled_lines[y]
        for x in range(0, C, 2):
            tile = line[x:x+2]
            if tile == '##':
                walls.add((x, y))
                walls.add((x+1, y))
            elif tile == '[]':
                boxes.add(((x, y), (x+1, y)))
            elif tile == '@.':
                robot = (x, y)
            # '..' is floor, do nothing

    moves = moves_str.replace("\n", "")

    deltas = {
        "^": (0, -1),
        ">": (1, 0),
        "v": (0, 1),
        "<": (-1, 0)
    }

    def is_free_x(robot, dx, walls, boxes):
        ry = robot[1]
        wx = set(w[0] for w in walls if w[1] == ry)
        bx = set(x for pair in boxes for x,y in pair if y == ry)
        cx = robot[0] + dx
        while True:
            if cx in wx:
                return False, -1
            if cx in bx:
                cx += dx
            else:
                return True, cx

    def is_free_y(robot, dy, walls, boxes):
        # Find the stack of boxes above/below the robot that must be moved
        rx, ry = robot
        to_move = set((b1, b2) for b1,b2 in boxes if (b1[0]==rx or b2[0]==rx) and b1[1]==ry+dy)

        changed = True
        while changed:
            changed = False
            new_to_move = set(to_move)
            # Try to find boxes connected in vertical direction
            for b1,b2 in to_move:
                # Check next layer
                connected = set((nb1, nb2) for nb1, nb2 in boxes if nb1[1]==b1[1]+dy and (
                                nb1[0]==b1[0] or nb2[0]==b1[0] or nb1[0]==b2[0] or nb2[0]==b2[0]))
                if connected - to_move:
                    new_to_move |= connected
                    changed = True
            to_move = new_to_move

        # Check if we can move them
        for b1,b2 in to_move:
            nb1 = (b1[0], b1[1]+dy)
            nb2 = (b2[0], b2[1]+dy)
            if nb1 in walls or nb2 in walls:
                return False, None
        return True, to_move

    # Simulate moves
    for m in moves:
        dx, dy = deltas[m]
        # Check walls for immediate next spot (robot dimension is 1 tile wide, i.e. 2 chars)
        if (robot[0]+dx, robot[1]+dy) in walls:
            # can't move
            continue

        if dy == 0:
            # Horizontal push
            is_free, next_free = is_free_x(robot, dx, walls, boxes)
            if is_free:
                min_x = min(next_free, robot[0])
                max_x = max(next_free, robot[0])
                ry = robot[1]
                to_move = set((b1,b2) for b1,b2 in boxes if b1[1]==ry and min_x <= b1[0] <= max_x)
                boxes -= to_move
                boxes |= set(((b1[0]+dx,b1[1]), (b2[0]+dx,b2[1])) for b1,b2 in to_move)
                robot = (robot[0]+dx, robot[1])
        else:
            # Vertical push
            rx, ry = robot
            up = set((b1,b2) for b1,b2 in boxes if (b1[0]==rx or b2[0]==rx) and b1[1]==ry+dy)
            if len(up) == 0:
                # just move robot
                robot = (robot[0], robot[1]+dy)
            else:
                is_free, to_move = is_free_y(robot, dy, walls, boxes)
                if is_free:
                    boxes -= to_move
                    boxes |= set(((b1[0], b1[1]+dy),(b2[0], b2[1]+dy)) for b1,b2 in to_move)
                    robot = (robot[0], robot[1]+dy)

    # Compute sum of GPS for all boxes
    # Use the left bracket coordinate b1 for each box.
    return sum(b[0] + 100*b[1] for b, _ in boxes)

In [4]:
map_str = """
##################################################
#..O.O..#..O.O.O.O.......##O..OOO.....O...O..#..O#
#.O.O.#.....O..O......O.O.OOO.......OO.O.....#...#
#OO........O.....OO......##OO.#OO.OO..O....OOO...#
#......O...O..O.#....O...O.....O..#.O.O.O...#O...#
#OOO.OO.OO...####O.#O...O..OO...O#.O..#.OO.O...O.#
#O#...OOOO......O#O#....#OO...O.....O.O.OOO.OOO..#
#O.O...O.#OO......#...........O...OO.O..O#.#OO...#
#O.....O.O.O.#...OO..O#....OO.O.O....#.........O.#
#.O.#O.O#...O#..O.O.#.O.OO..O..O...O.O...OO.O...O#
#..#..O.#.O..##......OO...........O.....O...#.#..#
#.OO#.........O...O.....OO.O...OO............O#.O#
##.O.OO.O...O.O..O..OO.#....O.#....O......OOO...O#
#..OO..O.O..O..O..O...O..O.....#OO.....O...#OOO.O#
#.......O.#O.OO..#......O..OO.......O..O.O....OO##
#.....O.........O...#.O..O..#OO.O...O.O.#....O...#
#.OO##O#..O.#..O......OO..OO.OO.....O..OO#O.O..O##
#.O....O...#O.O..O#....O#O...OO.....O....O.......#
#...OOO....#O.OO...#.OO#O...O#O..#..#.O..O....#..#
##....O.OO.O#..........O#.OOO.#....#...O#....OO..#
#.#OOO..OO..O.......#.OO.O..#O.#.OO....OO.O.....O#
#.O......#..O.#.O..........#O..O.OO...#O...O#....#
#.#....O.O....O.......O..OOO..OO...OO.#OOO.O.O.#.#
#.O.#.O.O....OOO...O...O#......OO##.O..OO.....O..#
#.OO.......#O.OO#..#....@O...O....OOO.O..OO......#
#O..#..#OO...O#OO.......OO..##..O.#.O.O..O.O..OO.#
#.#.O.OO.........OO#.O..O.O...OOO.O......O#...OO.#
#.OO...#..OO.##......O#.O.OO....O..O#.......O.OO##
#OO..O.O.O....O#.O..O......#...OO#.O..O.O..O..OOO#
#...OO....OOO.....O..O..O....OO......OOO.........#
#O...#.....O...O..OO..O.OO....O.O..O....O....O.O.#
#.....OOOO..#..#.O.OO.......O....OO#.O...O..O..O.#
##.OO...O.#O......#O.O#O#..O#..O.....O....OO..O.##
#..O..OOO...OO.OO.#O#.OOO......O...OO...O.O......#
#.OO.O..O.O.O#..O.#.O.OOO...O..OO.O##.O.#...#...O#
#........#...O##.....OO....O.O....OO#.O.O.O.#OO.O#
#.O..O.O.#...O.......#...#..O...O.#.........#..O##
#...........O.....O..OO.......#..#...O...#..O.##O#
#.OOO..O..OO....#..OOOO......O...##..#.......O...#
##..#O..OOO.#O...O..OO...O....OO..#..#.OO.O.OO...#
##O....#...OO#.O.O...O...O.O..O.......O.....O..#.#
#........#.O##...O#..O..O.#O........O#..OO#.O.O###
#....#.O............O..O...OO....OOO...OO...#....#
#.#O.....#...OO.OO.........#.#.OOO.OO.O...O#.O.#.#
##....#..OO...O...O..#O..O.O..O.O..O..O.OO.......#
#...........OO.O.O...O......O...O.OOO.....O..O...#
##O.O..O.O...O.....O.O.O.O.OO.#O.O.O..#.....#....#
##.O.......#O.#...O#O#O..O..#O..O....OO.O.O......#
#O...O.O.........O......O.......O......#..O.....##
##################################################
"""
moves_str = """
^>>^v><^>v>>^v^^^v<>>v><vvv<>v^>>v><<<>^<<v^>vvvv^v^>v^>>^<v><>vv^>^>^v<^v^v<<<<^<>v<<>vv<>^vv<^v><^^>^>^v>>^>^v>>>v<<^>^>^<^<><^vvv<<^vvv>v<v<<>v^><<v^<>v^v^^^>v^<^v<vv^v^>>^^<<>^><<<>>>^v^>vv><<>>v>vv<<>^v<>>v<v^<>><>>^<v<>^<^^<^v>v^<vvv^<v<v^>vv>>^^^>v><^<vv<>>><v>>>^v>><v^>vv<v>vvv<vv>>>^<v<^vv^<^<<v^^vv<^>^<<<v<v<^><v<<^^v^v<>>>^v^<<<vvvvv<<^><^v<>v>^v>>>><v>v^><^v^^<<><v^>^<v<>v><v^^^^<^v>^><>^^<^>^<<^<<^v>^v^^v><>v^^^<^^<>vv<v<<><>v^<vv>^vv><vvvvv<^v<vvvv^vv^v^>>vv>^<v>^^v<^v><^v^^v^^v>vvvv^v^^>^^v^^^v>v><>^<v<<^><v>^v^<vvvv>>v^v^v>v^<vv>>^>^>>^>><>^<v<<^v^<>^><^<vv>^^^vv>v>^^vv^>><^v<v<v><v<>>vv>>v<v<>vv<vvv^<><^>v<^^^<>^v<>>v>^>>^^v>vv>vv^><<^v<<^><<v<v^vv<<^v<^<>^>>^<<><^vv<v<<<v^v<<>^>v<<v<^v>>vv^^>v^><><><<vv<^><>v^vv><<v<vvv<>^<^^vv<<<>^^<>^vv>>vvv<<^<v>^^><<^><vvv^>^^>^<^^>v><<<<<vv^><>>vv<^<<^^v<>v<>v<<v>v^^v<>>><<^^>v>vv^^^<^<>^^><^>vvvv>vv^<v^<>^<<^^>^^^>^<^<v^<<v<><v^>v^>^>^>v<^<^><<<>><>>>^^<<>^v>^^>>v^v<^^<v>v^v<v>^vv^^^<>^v^^v<>v<>vv<^^<<>vv^>>^^^vv<^^^<><>^<<><>^v
<v<^v<<^vv^>^vv>^vvv<^<<>^>>^^>^v<^<>v^^<<^v<v><>>^<^<<v<^v<^^<<v<<^^^^><><v<^^^v>><<^<<>>><>^v>vv>>v^<>v^<^v>v^>>^v<v<v<>^<<^>>><><^<^v<<^>v^^^<><<>>><<^vv>>v^<vv<<>^^<<v>>v><>^<<>^^><<><>v>v^<^v^<^>vvv<>v>>>>>>^vvv>v^<^v<>v>v^><^vv<vv<v>>v<^>v>^>^<v^^^<<>^><v>^>><^<vvv>>v^v^>^<vv^<v^>>v<>>>v^>v<v<^^>^><^>><v^><^v^^<>vv>>^^^>v^<><^<><^^<v^v^<<<^<vv>v>^^>v<^<<v^>^>v<<<>^>><^v<vv>v^v^^^^<vv^><>v^>v>><<v<<>v<>v^>v><v^^^v^<v<^<>^v<v<v>v^v<^^v<v^>v<v^v^>><^>><v><>>^<><v^<<<v<><<^^><><<v><vvv<v><>>^>>>vv<^^^>^<<<<^^^v<vv^<>vv>>^^^^vv>v<^>><<v^^^v>vv^v><^v><v^<<<v<>^v>vv^^>v<v^<<>^>><<^v^^>^>>^^><<^<<>^>>v>v<^<v^^^v<^^>>^^^<>^>><<v><<v>vv<>>>^vv<<><v^>>>vvv^>vv><v<<>^^>^v<^^>v>v^<^>^<v<v<>>><vv<vv^<<>^v<v<^^<vv>vv^>v<<>^<>v<^>v^<vv<><<>^v<>>^^<^><vv>^^^<^v<<^>^^<<<vvv><^v<^^^v>>v>><v<><>><v>^<>>vvv<v>v^^<v<>>><>v>^>>><v<v<<>v^>>^>^v>^><^v>^<v>v<vv^><>v<v>^v>>^v><v<<^>v>^^<<^v<^<vv<>^<><vv^v<<v>>^v<<^>>^<^>^><v^><^>>v<v^<^v<^^v>^<v<^vvv<v^>vvv><^v<v<v<v^^>^<^>^^>v<^^^<^^>^v>><v^<<><>>v^^v^v^v
<<<>>>^>><><>^^>>^<<vv>vvvv<><^<^v^v<>>>><^<>v><^vv<^v>>v<<<^^>^<<><>^^><<^><<<<^^v^^>v<<vv>^>>^^^v>^<v><<^><<vv^<><^>^><vv>v^<<v<>v^>^<><<>>>v^>^>v<v><><<v<<>>^vv>>><<^^>vvv^<vv<v<<>><<^^v>>><v<<<^<^v>v^>v>>vv^^^v>>v^><<<>v><^<^v<<<^^<^v<<^^v<>^^<>^>^<v^^^^^^>v<<v>^vv<>v<>vv>v<>v<^^v><^<vv<^<>>^<v<>v^<>v><>v<^v>>v>^^><v<vvv^^<v>>v^><vv<<<>>^<v>v^vv>><^<<v>^^^>>>v><>>vv>^^<vv<vv><v^v>^v<>vvvv<>^v<>>^<><vv^><>>>^^^^>>vv^v^<>><^^><^^>><<<v<>^vvv^>vv><>v>v^<^v^<vv<><<<v<vv<v>^>v^>><<^>^>^vv>^<v>v^>v<<>>><<vv^<<v>v>^<<<<<<>>vv^^^<<v<<<^<<<v^v<^<v>>>vv>v<>>>^<>^>^v<>vv>>v<<^>v>^vv>vvv^<>><<vv<^^><<^>vvv><v^v><><<v^^vv>^<>>v^>^^>v>>>v>vv^><>>><<<<<>^>vv^<<^^<^>^<>^>v<v<<>v^v>>>^v^v>^v>^<<v>^vvv>>vv<>v><v><v^<>v^v^>v<><vv><^^vv^v<vvv>^<<^^>^>v^^>^^<>^>>>>v^<^>><>^v>v>v>><^vv^v^<^<v<<<v><v>vv<<<>>>>^v<<v^^<^^><>^^<^^v^v<>><><>><>v>>v^v^><^<^v^>>><vvv<>>><v><v^^v^><vvv>>^^>>>^><<<>v>^^v<v<<v<<><^<>^^vv^<^><v><^>^<>v><vv^^^v<<^<>^^>^^v<^v<vv<<<><v><v><v<<^^v<v>>^>^^<>^vvvvv^^^>><<^<><v>^>^<v>>>v
<^<^^<v^v>^>vv^v><>^>^>^^<vv<^><^v^><^v^><^><<<^^v>v>^>^><vv>^^^v^>^v^<v<<<<<>>v>>>vv><<<v>v<<><<v<v^>>^v<<v<<^vv<<^<^>><^^v>vvv>>v>><^vv>^<<<^>v<^<><^v>^<<v^<<v^>>v>>v<<^^v^^><^>v^v>^<<<<v^v^<>>v^>v^^v^<v^^<^>v<v>>v^v>>^^<>><vv^<>v<><^^>^^^^vvv>vvv>^><><^><<<>^v^<>v^>^^>>^vv>^<>>>>v<vv^vvv>><^^<<>^>^^>><<^<^>^^v<vv>^^>^v>v^>^^<<v>>>^<vv^vv^>v><>v><<vv^>>^vvv<<^v^<vvvv>^<^<^<^<vv<>>^<vv>v^>v^><<^^><^v<vv<>>^<v^<>^>v^<<<v^>>>>><<<vvv>v<^vvv>v><^^^<v^v^>v>^<>v^<>>^vvv<v^<v<^>^<v<v<>v<<>^<<>^^^^v<v<^v<v<<>^>^vv^>v>v>vv^>vvv>v^vv><^>vv^><^^v<^>v<>^>^<v<>^vv^^<<<<>^^^v^^^<>v<>^<>><v>>v>^>v^>^>^<v>^<^<>^<<<v<<^><^<v^>><>>v^^v>v<^^><><v<^<<^v>>^v^>^<<<vvvv>^><^><vv^v>^^^>v><>v>^>^v>^v^<^v>>v^v^><^^<^^<v^v^<v^>^^>v>><<v><vv^^><<^^><>v>><<<^<<>>v><>v^<<<vv>^<<<>vv<<v<>v<vv^v^>^>v^^>^vv^^><v<^><<>v^<v^>^v<v>v>>^vv<v<>v>>v^v^^^>>^vv^^><<<^<^v>v^v<<>^vvv<v<v^^vv<>v^><v><<^>^<^><>>>>>v<v<<<^vv<^^v^v>vv<^<>>>^>>>v^>^v^^^>^>^<>^^<v>>>><<><<vvv>vv><v^^<^v>>v<<>vv^^<<><vv<^>^v>v^v^vv^vv<><v<<<^<vvvvv>>
v^v<><v>v><><v>v^^^<^v>>^v>v^><v^v^><<>>>><><^>v<^^^>v>v><^^v<v^<><<<^v<><^<<^^>>^>^>><^>><<^^v^^>><>>v<v<vv<><<>>vv>vv^vvv<<>>vv<^^>><<vv<><v^<v^<>>^>^^v>^>>v<<<<>>>v<^^v>>v>^><>^<v<v><>v>>^>vv<^^v<<>v>>^<^^>><^<v>>v<>v>^^>v^v<^<^^<v>v^<^<v^^>^<<^^>>v^v^^v<vv<<v<^v<>^v^<^^<>^^>>^<^v><>v<v><^^^<vv<<^^v^<<^v^^v>^v^<<^vv>v>^v^v<^v<vv>v<^><>><^v><v<vv^<>v>>^^^>v>v^v^>v>><>><<<vv<v^<^v^>>>v<v>><v>v^>^<>>>>><v^<^v>^<^<^^>>><v^><v^^^<>v^^>v^vv<>^<<v>^>>^<^>^^>^^>>^<v>v>><<>^>^>^v>><^^<^><v^v<<^>>vv>v>>vv<v<>^><><<<><><<^<>>>v>^^v<<>v^>v><<vvv><>>v<<^<>><<>^><>^v<<^vv<^<^<>>vv<^v^>v><v><^v^v>^<><>^>>>v^><>>^<<^v<v^vv^><<v^>v^^><>v>><v^<<v><^>>vv^>^><^v>v^><>><>>>v>^^>^<v<vvv>vvv<><v<vv>>>>^^^><<>^<^^<^vv<>v>^>>>vv^>>>v>vv<<>^vvv^<><><>v<^<>>^<><><>v^vv>^v>vv^vv^^<^<^>>v<^<v<^<vv>^<>><v<^><<><<^v>>vv^>v<<vv>^<>^v>^>v<<<<>>^>>^^^^v<<^^><v^v^^vv<v<^^v^v^v<v^^>>^>v^>^^><^<^<^v>^^v>><>>^>>v>v^^^<^^>>>^<v<<<>^^v^v>>>>^^v^><>><v<^vv<><><v^>>v<<>>>>vv^>^^><><v<^>vvv>^>>v<<<>><^<^^^^^<^>^^^v<><^^^v<vv
<<^<^>>^^<<>v^^<<^<v^<v>^>^<vv>^<v<v<>><>^<><^<v<<^<v^<<<vv^vv<>>^^^<v<^v^>^vv<>^^<><^^>>v^v<><^>v^><^^^>v^>vv<vvvv<<^^>^v^>^>^^^v^<<v<vvv^^v>^^v>^^>^><><vv^^<v^<>v<vv^^^>>^^<<v>><<<v><<><^<<vv<^<^<v>^<^<v><<><v<<<>^^^^><^^v^<>v>v^<^v>^^vv><><vvvvv<>><v>^vv^^^v><<<<^<<<^<v^>^>>^v<<>vvvv>^>^v<<v^>>^>v^>v^^^<v^v<vv<<><v^^<^^^v<v<v<<v^vv<<<^^<>^vvvvv^<vvv><^<^^^^<<^>v<^vv><<<v^>vv<v^<<^^^v<v>^v>>v^>><><<v<<v^<^>><^<vv^v^<v>^v<<>v^<<>vv<>>>^<v>v^^<^>^v<>^>v^v>^<>vv^vvvv>v^<<>v<v>>><^^<<^^^<vvv>^v<^>^vvv^<>^v>><v<>v^<>^v^^^v<v^<<>vvv^^<v><vvv^^^^>>v^^<^>^><>^v^<v>v<^<v<<<><vv<v>v><^>>>^v<v<^<^^v^>^<v><^^>v><^v<<<<v^v<v^vv^>v>>^^<<v^<>^v>^>v<v^v><^v^v<v^^>vv<>^>>^v<^<><><>^>v^v<><>^v<^>^<v^<vv^^>>><v<>v<><<<^^>vv<><<^vvv<>^^^^>vv<<><>^^>v>^^^<vv>vvvv<<^^<v^<^>^>>><^>v<v^^><^^vv><v><<v<^<>v^^v<<vv^<>v<<<^<^><^v^^<^v><><<vv><v<v^<<>v>><<><>^><><<>>>^v^vv<vv^v><<^^<^^>^^^<v<<<^v<v^<<<<>v<vv^vv^>>>><>vv^>^<^<v<^v>>><^><<>v^v<>^v^>>v^v^^v^>vvv^<<v<^v<><>v^><v<v>>>^v<<>v<<>v>v>>^>><v>v^><vv<vv^<<v
><^>v><v<<<><^>v<^<^v><v^<v^>^<vv^^<^^<vv>vv<v><<v><^^^v<^v^<<<<v^<<^^vv^<^^^v<v^<><>>v><v<^><>><^>^><^v^^<^<<>><>>^v><>vvv^^v<vvv<>v^^>^v<<<><v^>^>>v>^<vv<<vv^^>v>^^<^>^>v><v^v>^<>>><^^v<>^^<>^v^v><>v<>v^>v<^>vv^v^>^<vv^<v^v>v<<v><^^<^^<<^^<^<<>>^<v^<>>>v^^<^^v><>><>>>><<^><>>^<v^v>><^<>v>>>^>><>>^^<v^^>^v<>>v^^v>vv^v>^<^^^<>^<v^^^<vvvvv<<<>vv<v<>v>v<^<^^<v<^^^<>^>>v>^<><v<<><<^>^^v<>v^<^vv<<<v>><><^>>^>v><>v>v>^v>^<v>>^^^<<<^>^^>>><^><^v^v<^>^^^v>v<>v<<<^<^<><>><>^^>>^<><v>^<^v<vvv^<><v>^<v>v>v<<>^v>v^><>><^v>vv>v^<><^^<v<v<vv<><^^v>^^^><<>v<^<<><^^><<>>v>^><v>^v<<^<vv<<^<vv^^^<vv>>v^v<^><>^<v^vv><><<^><^^><v^^v<^><>^v>><<^v^>v^^v^^><<vv^<>^<>v^>^^>>v>v^^vv>>><^>>^^<v><<^>^>>><>vvv^<^vv><^>^^<><<>v^^v<^vvv>^>>vv^^<>v^^>^^<vv<<v^>>^<<v<v<^v<<^<^>>^^v<>v^<v<<<v><^vv<^^><^v>^<<<vv>v<v>>vvv<><><vv>^v^v^v<^v><^<v^>><<vv><<vvv^vvv^<<v>^v>^^><^^vv^^<<>^v<>^^v<^<v^^v>v<>^>^<^^>>>v><>^<v<<v><>^^>><vv<^^^^^v^^v^^v^<^>vv><v<<<^^^<>v>^vv<>^<^v^>^^<>>^<><^<>><>^v<^>vvv^<^<<<<^v><vv<v<><^>><<<v^v>
<>vv>>><>>^>^<<><^>><>v^^<^>>v><v<v^>>><v>^<^<>^vvv<>^^v<v<^<<v>v^v^<><^v>>v^>v^v^v>><<v<>>v<>>^<><>v^<^^>v^>^v^>v><^>^^v>^v<>v^<>><><v>v>><<><v<v^^v^><<^>^<vv>v^<^v>v>^<<<v<^v<v^v^v^<<v^vv^^<<v^v>v<vv^>^^vvv^^^^v>^<^<vv>>>^^<^>^><v^>^^vv^^^vvv>v^v^v<^<^vvv<><<>>>>^<v^v<^v<>vv^<^<<^v>><^v<><^>^v^>^^>^>vvvv^^^>><^^<<^v>v<>^^^>v^v>^<vv<<<<>^>^>^>^^<<^vvv^<<<^><><^>><^>>^><^<v>^^<<<<>v>v><><<vv<^>^>v<v<>v>v>^^vv^>v>v>^>^>v>^^v<><vv^^v^^v>^^^<>^><^^v>v>^^>>^<<>vv>^vvv>^v^>v<vv^<<<v<<v^<v><v^v^^>v>^><vv<v^<v><<^<^<v^<>^^<v>^>>^^><>vv^><>v>>v>^^v<<v^<>>^v>>v^>v^>^>vv<>>>^<<v<><<<<^vv>v>><^<>^>vv>v<>^<><<><<v>^^vv^<<v<vv<^^>><<<>>^>>v>>>>>^<vvv^<<>vv<^<vv^^<^^v^<>^<>^^>^^>>v>v><<^<>^v<<<<v>^<^v^v>^^v>v<^>>>vv<>>v^^^^><>vv<v>><<v<<<>v^v^>vv<<v<^>v>vv<>^>^v>^<>>>>^^<><^^<<v<v^>^^>^vv^<v<v<>^vvv>v<^v^^v<>v<^>>vv^^^^^^>>vv<><vv>>v><><v>vvv>>^>^^vv^v^^^>^>^v^^^v<v>>^v<>^v^^^>^v^<v^><^vv<v^v<>vv><>vvv>><^v>v<v>^vv><v<<>v<v<^v<v>vv>^<<>v>><>^<v<><<>^<>>^^><<vvv^>v^^v><<<>v<<^v^^>^<vvvvv><^>v^^<<^>>v
v<>>^^<<>v>^vv<><>vvv><>>^v><v^>vv>>>>^<v<<v<v<vv<>><^<^v>vv<><<<<v^<^>v<^v<>v<^^>>^v><><^^<<^^<<><<vv^<>^^vv>v^<v<<v><^vv>>v>^<>>v>v^>><>v>v^>v^^>>^v<<v^<>><>^^<<v^>vv^<v^^v<v><^^^v^<^>^>v<<vvv<v^<^^v>>^><v^<<vv<><^<v^<><>^><^>>^v^vv^<<vv>v<>>^>^^^v^<>v^>v^^^>v<><><<^><v^<<<v^><>^<<><^v^<<v>^<>>^<^v<<v<>v^><v<v<^v^vvv<^^><<^<><vvv<^^v^>^v>^^<>>><v<<>^^<<<v><v<>^v>^^<v^><v<<<><<v<<vv>v><>>><^>v><<<^^v>>vvvv<<v>>^^^^^<^<^v^<v^v<<vv>>v^^>vv>^^v<vv>><>><^<>><v^>>^>^<vv^>v^>vvvvv<^^><>^^>^^v>>>vv>^<<v^v>>v^v^<v^<<^v<>^^^^<^v><<^vvv>^<v><v^><>><v<<>^^v>>>>><><>>>vv^<vvv><vv>v^<>^^v^>>v>>^^<<>v^<>v>^v>><v>^>v<<><<>^><vv^>v>>v>vv^v>vv<<v^>v<>^<v>>><v>^^^>vv>^>vv^><^v^v<vvv<<v>^<^vvv^<>v>^^^<^v^>^>>>v^<><>><vvv^<<<vvv>v^<<>v>^v<><v^vv<>^<><><^>^^^^v^v><<v<<vv^v<v<<<<v^v<v>^^^><vv^^<<<<>v><<<^^>>^^^^>^vv><<v>>>v^>^vv<vvv^<<<^><v^<>^^<>^<<<vv>v>^^>^vv<^<<^<<^<><<v^>^v<v>>>>v>><^<v><<<^>^<v<^>v<vvv>v^vv<v<^^>><v^v<v<vv^v^^><<^><v<>^<^v^><v^>v^v>>^vv^<<^v^<>v<^<^^v^<>^<>^><^>^^^^v<^v^^v><<vv^<<<<v
>>>>v<>>v^>^>>^<<v^<<<^<>v<^^<v>^^>^^<^<<v^<<^><>>>v<v>><^^>^<<<^<>^<v<^>>^>v><<^v<<<v^vvv<><><><<<>^<><^v<><v^<>^v<vvv^^<v<v^>v>>v<v^v>vvv<^v^^^^^v>v<^^^vvv<v>^>>>^vv^>><>^>^<>^^v<^v<v^v^><^^^^><^v^><<<>><<<<>^<^>v<<v<<<vv^^<<^>>v>^>><<<<^><^v<>^>^<>^>^<>^<<vv^>^<>vv>^v>>><^<^^v^>>^v>vv>v^^>^<<<<v><vv<<<^v<<vv<<^^<<>^vv><>v^><<v<v><^^><^><<<^^v<<vv><<><>v<<v>^>vv>^<^>^>><<<^><>vv^<>v>v^>^v<<^><<<v<^^^>^v<^^^>><<^v^^<v^^<^>v<v<<vvv<>^<^v<<^<<><>^<><>^^vv<>v<vv>^vv^<^v>v<<v><>^><^>v<<<<^>^>v<vv^>^^<^<>>^v>v>>v<>v>>^>>^<<^^<>><<vv>v^>v>vvv>^^<vv>vv<vv>><>>v>^>v<><^><<^v^>^><>vv<^<>^v>>vv<><^v>>^>v>v^>^^v<>>^^>^^^v<v><>vv^vv^^<<><>^>>v<vv>><^>>v>^<^<>^><v>^<>^v<>v<<<<v>^vvv^<^>v^vv<><<><v<>v<<v>><>v<>v>v<vv>^vv><^<^^v<><<><^^v<<>v<>v<>^><v^v>>v<><vvv^>><<vvv<>^<><^v>^v^v^><>v>vv<>>^^>vvv^vv<<^v<>^v^vv<v>v>>>><v><v^v>^^vv<<>v<^<vv<^>^<vv>^<^<v<>>v^^><^^>>>^v><<v><vv<<>^vv>vv^^^^v><^^v>>v<>vvv>^v>v^<v^>vv><><v<<vv^v>^^v<v>vv^^v^^<^^^vv<^^>>>vv<><<><><><v<<><>>v>^>>^<<>><v^>^<^^^>^><^vv><^<^
vv<>><<>vv^>v^^v>><^>^v^>v<<^<^>vv>^^v^><><v<^<><<^v>><>vv>v^<<><vv<<^vv<v^<^vv<^^<^>v^v^<><^>v><>^<<>^<v^><v>^^<^>>>>>>v><^^><v^^><<^v^v>v>>><^^v>>v>><<<><<v<vvv<^^<<>^^<vvv<^^^^^^<<><v<^>v>>>^>^^^<<<<<<v^<v>v>>v<^<v<^^vv<^^<<^<<><v<<<<v<<v^^^^^^v<^v<^^^>^^>^v>>vvv>v<>>v>v<^<v^<>>v<<><<<>^^<v^v>>^<v>>><v>^<vvv<v<v><>^v^^<>>>>v<vv<<>v>>>><^v<^^<v>>^v^<<vvv>vvvv>^v><v>><<v<<>^<^^^v>v<v><^<^<>v<vv<^>>^>><<>>>vvv^<>^v^<>^<<^<v^^<v<<<vv>vvv>v^<<><>^>v>v<v<<<>>v>>><v^>^v^v^v>>>^v^<v>^>^^><>vv<^v><><>v>^<^v>^>>^<<^>^^><<^v<^v^<>^v<^^>vv<<vvvv>v>v^<>^<<v><^<vv>>^<<>v^<v^<<v^^^v<^<<^<>>vv<<^><^>v<>>>>>>><v<^^>>><vv<v<>v^>><^><^>v^>^><<vv>>>^><vv<<v>><v<<<><>v>^><vv^>>^^v<<v>vv^><^v^><>^v^^v>^^<>>^vvv<<v<>v><>vv>^^>>v>^>v<^^<vv<>>v<v<vv^^<>>v>v<>^v<^vv>^vv^vvv<^v><>><vv><<^<v<>>>>>>vv<>^v<<<^<<vv><^<<<^^<v^<>^^<v^<v>>^<^<<>>>^^v<^>^v>^<^vvv>>v<^<vv<^>^^^>><>^<>v^><>>^^v^<v<v>^>><v^^>^>v<v>v<><><v>><<v^<vv>vv<^>^v^^v<v^>v^><<>^v>^^<^v^><v<><<><<v>v^>^vvv>v<><<>^^<>^<><>^<>v^><v^><><><>>^<vv^><v>
^v^><<>vvv^v>><>vvvv<<vv^v^v>><>v<<^v>^v^<^^><><>^>>^^v^v>v<^<>vvv^v^>v^v^><>^<><<><v^<^>^>vv>v>><>v<><><v^>^>><<^<^<<<^<>^><v^^v^<^<v>^<>^>^^>^^>^><<><>^<v>^>><v><<^<><v^v><v<>>v^>v^^>vv^v^>v^><vvv<^<v><^><>>v>>>v<>^>v^v^^^v<<v>><v>>^^v<vv^^^^v><<<^v>^>>>^v^>vv>^^>vv<^^^v<<<><<v^>>>v^<v>>v>vv><><<v><>^<v<<v<>v^vv^><>>><vvv><<>><^<<<>v><v^>^<v<<vvv><^<>^><vvv<<<vv^^<v<<>v<<<^>>^^v><vv>v><^<><<^^^<>^^^>v^v>v><^<<^v<>v^>^>>^^^<^<>v<^^v<^>^^^>^<^v>v>^^<v<^^v<<^vv>v<v^<v<>>v^v^<^>>>><<^<>vv<v<<v^>>>^v><^^>>v>^v<v>^>v^>>^^vv>v^v<<^^^^v<^^><>^vv>>^<^<>v<^^^<^v<^<><<^v>v^<>^v<>^<>v>><vv<^<^^v>>><>^^^v<^>>>^v<v>>v<>>><vv>^<^><v<v^>^<^<><<^v>><<<^>^vv^^v>^vv<>v<<<^<>vvv<<<<^<>>vv>v^<<v^^><<>vv<vv>^^^^<v^<^>><<^^v<vv<v>>v^v^vv<><vv^^>^^<v^v>vv^^v><<<>>^^<^<v^^>v><^>v<><v><^>v><<^>>>>>^>v^^v><^v>^>v^>^>^^>^^v<^>^^>^<<^v^^vvv><<>vv^>^^vv>^vv^v>><>><vv><<><>>vvv^^vv<<><v<^^^<<^vvv>^vv>^v<>^^^vv><^v>v^<v<v>vvvv>>^^^<v<<^>^<>^<^><<>v^v^v^<v^><vvvvvv^<>><<<>vv<<v^^v>v<v<^^<<^^><^^<<<<v><<^<>>v<>><vv<<
<>v^^<>>>>v^^>vv>v>v^^^^^vv<v^^v<<>><>^v<>^v>vv^><>v>><><^<^>>^^v^<<^><^>>>>>>>v><vv^^v^>^<>>^vv><vv>v^v>^^<>>><v^<<^<>vvv^><><^<<<^<>^>>>>v^^^>^^^<<>^<v<>v<><^v^^v<><>>^v^>^><<^v^^>v>>>v^>^>><^<^v><^<^^<^>v^^<^v^<^^^><>><^^<<>><^v^v>vvv>vv^<<^vv><<><>vv<>v>^>vv<>v<^^<<vv<^<>v^vvvv><v>^<>^v<^vv<v^v<^^>v^^<<vvv<^v>v><^>>^>^^<>vv<><^<^<^vvv<v^>^<>^<>v>vvv>v<<v><<<<>vv^v<>>^^v>^^v>v<>v>^v>>><<><<^v^^>^^v^^v^v<v>^<v>><v<^>^<>>v^^><v<>v>>^>^<><v><><v^<<^v^v^^^^<v^<vv>>vv<<<><^vv^<^^v>v^^>v<>><><><<>><^<<<^>v<^^^>^^v>vv<^>><><v>>^v><^<>v^>>^^<>^vv>>^^^^v<>><^<><>v>^v<<>^<v<^v^v^>vv>>>>>^>>>vvv<<<<^<<^<>><vv<v<<vv<v>>^<v<<<<^><<>v>v>v>^<<^^^<>^vv<>^v>vv>>>vv^^v<<>vvv^>^<<v>^v>>vvv>><>vvv>>^^v<^^v>^v>vv>>><vvv><vv>>v<<^v^<<<>^^>vv^><^v<>vv<<^<v^^v>^<<v<^>>><^^<v<^v^v^^^<<><^v<>^<<<v>^v><<<^v<vv<v^<v^^vv^<><<<>>^<><>^^v<>v^><><^^>vv^v>v>^v>^<>>vvv^>^vv^><><^^v<<^vv>^^><v>^<v^<v<<vv>^><^<v<<^vvv<><>^>^<^vv<^>vv>vv>^><<<>v<<^vv<>vv<^>>v<>vv^<v>^v<v<>>^<^^v^vv<><>>v>><vv>><v^<><>v^>v^><^^v<>v<^v^>
v>><^v>>vvv^<>^v<^<^v^v>>^^vv<<v<v>>v<^><<<v>^<<<<<^v<vv^v<v>v<v^>>><^>^^^vvvv><<vvv^><v>><>><v<>>^v<vv^><v><^^<>^>v<^<>^vvv^v^^<<^vv<<v^vv<>^vvv^<v<>><vv>v><>^><v<>^<vv<v>vvv^^>v<v<><vv>>v^^><<>^>^^>^>^^v<<v<>>>>^>>>v><v^<<<<^^>v<^^^>^^vv<><vvv><^<<>>>^<<vv^<<vv^v^<^^vv>v<^^<>^><<>^<>^>>><>v<>vv>^><>^<^>>^^>vv^v>>v>^^v><><^^v^^^^v>>>><>>vv^^v<^v>v^>^<v<<>v><vv>>v<<>^<<<>^<v>vvvv>v^<v<<<^^vvv>^>v<<>^^<<>v<v^v<>vv<^><vv<>^<v^><v><v^^^v>^vv<v>^^v<^^>v<><vvv<><<<>^vv>>v>^<v><^vv^>v^>>v>v^^<<v<>>v<>v^><<<^v<v><v><^v^><>^^>v^<v^<>v<^>^<<^^><><v<vv>v<>v<<^v><<><><^^^<v^^<v^<^vv^<v>v><<^<vvv><>^<<^^v^^><>><><>><^<><vv<>>^<v>v><^>^vv<<^vv>>><>v^>^vv>^vvv<^^>v>^v^^^^<v^<v><^^^v<v><<>^^><<<<v<>v><<>^>^vv^<^>^<<<<<vv<<^<vv<<<v>^<<>^>^v>^>>vvv^>vv^<^v<>>v^^<v<^v^^v^v^v<<>><v>^>><vv^^<^>>><><^>^v^^<^>><<<>^^>v<><^<>^^v^>^v<v>v<<<^<><v<<>v<<<v^<<<<^^>^><vvvv<^^>>^v^^v^<^>><<><><v<>^^v^vvv<<<v<v^^>^>^v>>>><<vv^v<v^>>><v>v><>vv^<><>><^v>^>^v>v<^<>>^vv<vv>vv^^^<<^v^<><>v^vvv^>^>^^>^>^^><<^^vvv<<vv>^<><
^vv><>^<v^v>vv^v>>vv<<<v^v<><>^v<<v>^>><><^>vvv<^v>>>vv><<^^<v<^<><^vv^^^^><>><v<>>>>vvv^>>vvv<^<<>><>v>^vvvv>v^vv>v^<>v^^>>v<>v<<^vv^^<v>^>^^<^v<<^v^vvv>^<<v>v<<^<><><v^>v>^<^<<^>>>^^^<v^^<v>>>>v>>v^<^^><>><^^<^v^>^>>v^<v>^><<^<^<<^v<vv<<>>^v>^^<^^>vvv>v<>v<>v<<>^<^<<><<>v>^^>v<^vv^<^<>>vvv>><<vv>>vv>vvv^^^^>>>^^>^^>^><<vv><>>^<^^^<v<^^^>vvvv^>^>><^^>><vv>v^<>v^<v><vv^v>^<^>^>^^v>^v<<vv<><v<<v^^^^<><v<>^>>^><^<^>vvv>><^><^<<v^<>^v^^^<>^^<^v>>>v<v>v<<<^>>>^>^^^^^<<v>^v>vv^^>><vv>v<^^v^><^^<^><v^vvvvvv<^v<>^v^v>>v^^^>v<vv>>>>>>>>^>><<<<><<<v<<><^v>^><^^<<vv^>v<v^v<<<^>^vv>><>v<><>^v>v<^<<<<<^>vv^>vv^v>^v<^v^^>v>^>>v>>v^v^v>>v<v^<<^><<>vvvv<><><v^>>^<v><<vvv<><><v>>^v<<v^<>^^v>><^v>v<<><>^v^^v<^<<>>>^^>^<>^v>^v^><>v>v<^>>vv^<><<vv>^v<^vv<^^v<>^^vvv^v<<<v^v<^<^^<v>^vv>v^>^>v><>v>>^>>^v>>v^v<^^^v>>v^<>>v>><>vv<v>v<^<>v^>^><><v>v<>>>>><vvv<<<>><vvvv^^<v<>>^^>v^v^>^^<v^>><v>v^^><v>v>^^>^^v<^<^<<^<^vv^>v<v^<^^v><v<^><^^^v<vv>^v<^^<>><v^^v<v^<<^>^>^<<>^<^><>v<vv^v^^>^v<^^^>^<><<>^^<>>>v<<v^>>>
>v^^^>v<>^v<v>>^^^<v<>><>vv^<v>^<><<vvv>>>^<>^vv^<<<^>v^^<>><v>vvv>>vv^vv<^^><^^<<^v^^><<^^><v>v^>><<>^vv>^<><<^^<<vvvv<^vv^^<^v><v^^v<v^^v><<<^^<><<<^<^v><>>>v><<><v<^<><^vv<v<<<>v>>^<v^>>^^<<>^<>v^>><>>^^^vv^v<>^>^v<^<^>v^<v>><<<v<^^v<^^^<>><v^>>><^><^<v><^v^>><v^>>>^><<>>^<^^>v<<>v^^v^<<<>^<>><>^vv^^<^^>v<><<^>><^v<>><>v^v><>^<^>^<<v>>v^^<v<v^<<<><<<^<<^^v<>v<^<v<v>^>>>v<v<^>^<<<>><^^^>>vvvvv^><<<^^v>^>>^^><<>>><<v>>v^<><><v>>v>>>v>>>vvv>vv<^<>^><><v<v<><v^<<^<^^v<^<^v><>><>v>v>v^<>><v><<vv^v^^^<<>vvv^^v^v^v^<<^<>^v^^<vv^v<>^>^>^>>v><<^<>^^<^<>v^>v<>><>v>^<v^^><>>^<<v^<><>^><<<<<<^^^>><^^>^^v<<<v>^<<<>v^^^v^^^^vv><>><v<^^>>>v^<^<<>^^>v^^<v>>v^^v<<^><<^vv<>v>>vv^<v>><v^v^<<vvv^<><<^<^^><><v>^<^^<v<vv<^^^^>^<>v^^vv<^<vv^<>^^<v^^<^>^><^>^v>v^<>>^^>><vvv<>v<v^^>v<vv><v>^><vv<^>>>^>^<>v^^v>^>>><>vv>v^vv<<^<<v<^<<v>><><><v<<^^^vv>>>v^v^<>^>><>>>vv>><^vvv<v^><>v>>v<<>^vvv>>^vvv^v>>>>^v><^^^>^^><^v^v^<v<^<v<^^>vv><v<^v>v<^^vvvv>><vvv^<^v^<v<><><vv>>v>^<><v<<^vvvv>>>v^>>v<<>^^<<><^^^<><v^v^v
<><>vv>v>vv<<>>v>><^<<^>^>><>^>v<^^^^^<^>>>>v>^><^^>v<<<^^v>>><>>^v<v><>v^^>^^vv^v<>^v<^^>v<^<<>^><>v<<>^^<v>v<><^<vv<v^v<vv<<^v<v<<<><>>^^>v<<>>v^>><>><>^vv<<v^vv><><<<v<v><v<<^>>^<>vv^<v^<<><<>>^>>v>>>v><>^>v><v<v>><>^<<^<^^<vvv>v<<^>^v>>>>^>>^>>v<^<vvv><>><vvv<>^^^^^>v>>>>>><<<<v<<<>><>><>>vv<><>v<><<^^<vvv>^<^<^^^>><vv^>>^v<>v^^v^^^<vvv><<<<<v<v<>^<>>vv^><^<^^^^<>v<<vv<>vv>><<^vv>>>>^vv^>><v>v^><v>v<^>v>v^v>><^<>v>>v<>vv^<<vv>>>^vv^>v^<>^^v^<>v<v^<<>><v^>^^v<^>>>>>v>>><>^><^v<>>v>>v<><><<>^v^<<^^v>>vv>v>^<>^vv>^^>^v^<>>vv^^^<>^<^^<<<^vv><<v<^v>v>>>^v>><>^^>>v<^><<>^<^^>^><v<v^>>v>><v<v<>^<vv<<<<vv>v>vv>>><>^^v>^^>v<vv<>v<v^>^vv^>^<>><><vv^<^^v><v^^<<v^^v^>><>>v<>v^<<^v<>>^v<<vvvv>v^>^>v><<<v>^>^>v><v^><<^>>>^^>v>>>^<<><v<^>^>>^v>v><v>^>^>vv>^>>^^>v><>vvv<>v^^<^^^>^<><^>>>vv<<><v>v<>v<<^>^^>v^^<v<<<>vv<>>vv^>v>><vv>>vvv^>vv<<<^>^^v>^^<v<>v>>^v>>^v>^v>>^>^<^v<^>><vv^<>>>^<v^<vv><vv<<>>v^><v^<<v<>^>^vv^<v<^>>><<<^^>>vv><v><><><^>v>vvv^><<<<<>v^^<^v<^>^>^^^^v<<^^^v^v<<^><^^<^>^v>><>>v^
>><v<^^^<>>^>^v<>>>^v>>><><vv>><^^^^^>><v>v><^^^<<v><vvv>vv<><^v<<^>v<^<><v<>v^<v^<^><^v>^><<<^v><><^<<>vvvv>^v^>><><<>v>^^>^vvv^^^<<v^>^^^<<v^^>^v^^vv<>vv^v<><v<>^v<><v^>^^<^><^>v<><^v<<><^vv<<<<^^^>>>^^^^<v^^^v^>^v<^v>v>v^><^>v<<>>^><><>^<^v>><<v<<<^^>>v><<v^^><vvvvvv<v^<>><>^>>>v^>v<vvv^<^^>^>^v<v>v>>^>v<^v>^>^<v^^<>^><<<<<>vv<^v<^v<^<^><>v^<<<<v>v>><^<v^v<v^>^><v>><<<>v^><<^>v^<>^<v<<v<<<>^<<>vv<>^^v>>v>^^v^><><vv^<v<>>^>><^^>>>>^<^>>>><vvv>^vv<>vv^<v><^<>^>^^>^<<^><<>>>^>>v^>v>^^^>^vv><>><vvv>v>>v>^<v^v>^><<<v^>^>vv<>v^^^>^^v>>vv<>>^v>v>^^<<vv><v<^>vvv^><<v><<v^>>^^^^>vv^<<><<>vvv<<v^^v^v<<v>^>>^v^<^>v>^<>>>v>v^^v<<v><>><<><^vvvv<<>^v>><^><v^vv>>>>^>><v<>>><<^^>v^^><>v>vv^>>>>>><vvv<^>>vv<>^<<v>^<<>v>>v^v^>^^>^>vv^v^>^><v>>v<^>^v^>^<^<v<<^vv<^><vv<^<<>v<<vv^>^v^>>v^v^<<<>^<^>>^<<v>>^>vv>v^^>v>>>>><^^<><>v<^^<v<v<v<v>>^<^<<>^vv<vv^>^v^>v^<<v<^v>^>vv>^<v^^^^<^<v^^<v<^<v^>v<^><>><vvvvv^><v<>v>>^>>v>>v>^<vv>v><>>^>>^<<vv^^><^v<<>^<v^^v>^^v>v><<v>^v<v^<>^>v^^>v>>^<^^>vv>v<>^v<vvvvv<^^v
<>>^vvv^<^^v^v<<v<v><><>^^>v<<v<<<v<<^<^^>vv^^<^>v><v^^^v>vv<>>>vv^^<><<v<>^v^>vvv><>v^vvv>>^<>>vv^v>><vv<^^<><>>v>v^<><v><^<><^v>v>v><^>^^^v^<^<v<><><vvvv<<>v><^^<><^v^<^>v>v>vv<vv<<><^<^><<v^v^v<v<><<^^vvv>><<vv^<>^<v^>v^^>^>v>vvv<<><^>^^^>v<^vvv>v<^^>v^^^>>>><vvv^<<<<^<v>><><^<v<>v>v^^^^<v><v<>v<v<^^v<<^><^v<v<v^vv>^>v^^v^^>>><^vvv>>>^^^>>v<^<v<>^v<vv<^^<<v<v><<v^^v<^<>>^v^>^<><v<<v<>vv^^^v<v<<<v<><>>v>v><<<>>^>v<v^^v>vvvv><><>><^^v>vvv^v>^v>^^^^<v<^^>v<>>vv>><vv^v<^>^<^v>^v>vv<><^>><^<^^^^<^>^>vv<v<vvvvvvv<^<><^<<<vv<^<>^vv>^^<^><>>v^<vv<<<><><v^^>>^^^v<<<^>^v^<vv>v>^v^<vv^^v<v>>v^<<^>^><><<^^^<^^>>^^>vv<v^v^vvv>>v<v^v<<>v<vv><v^v<<>>vv^^^>^^v^>^<><>^v^<><vv<^v>>v^v^^>^vv>vv^^><v>vvv<v><<>v<v^vv^v^><v<^v<v<^>^<>v<v^v>v<v^<v<^^<>^>>v<<>><>>>v^<><<<<<^<v<<>><><v><<<<^<^^<^>>^^><<^v><^>^vv><>v<v>v<><v^<>v<vvv^^<<<>^^v><>>v<v><<^v<>><v^v>><<<v><v<vv^^^vv>>^^<<><<^>v^<v^^^>v><v<>>>^vv<^^>^<>^v<v>>v^<>^v<>^^<<>v<>^^><v^<^^^<<^>>v<><v^<>>v<v^v<>v>^>>^v<^^v<<^><^v>^>v<v<<v^^v>^<>^^v<v>v><<
^<<v><>><<^vv^^<<<>v<^<><^><<^><^<<<^<>^^<>><v^v^vvv^v<^<>>^v<<^>>v<>v>v<^v^v^v>^<^v<^^>>v^<v>^<>^<<<>><<^^><>>v>^v^>><^v>^<<vv^v<<v^<<>^^>^<<><<^v>>^v^v>^<<^<v>v^v>>v^>^>v>^v^v<v<^^>>v><^>^^>^^^v<vv>^>^^v<^^>^><>><<>v^vvv><<><^<vv^<>v><v^<^<>>v<<>>v>>^><v>v>^^^<^^><^>>>^v^^^<>><>>>v^vvv^v<vv^^vv^<v^<<<^^>^v>v<v^v<>><^>>v<<^>^>>v^<<^>>><vv><^<><^>>^>>^vv^<>v<^^>v>>><<>><<^<^v>^>^>^v^>>><^><<><^<<><<<>v^^^^vv<>vv<<^>>^<vv<v<^v>^><v>>><^vvv>^<v><>^v^^<vvvv>>v^^v<>^^>^<>><><v<^^vv<^<<<><^^<^^^^<vv<v><^^<>><>>^<vvv>><^^>^^><^<><v^<><>>^<v<^^<vvv<>^^<<^<^^^>><^^>v<v><>^^^^>^^^><v>><v^>v>^^>^v<><<^<>vv<>^^><^vv<v<>><>>>^<<<v>>><>v>^<>^<>^<v^<<v^^v<<<<v^<<>>^>v>>v>>>>^<v^vv<<v>^><^^^^^><<^>^^<<<<v>v<^vvv<<v><v>><vv>>vv<^^<^^vvv^^^^v<<<^<^>v^vv<<^>vv>^<v><>^>vv^>v>v<>^<<>^^v>^^v>>>^<v><v<vvv<vv<<vvv^^<<<<^^v>vv>>^><v^^>>>><>>^v^^<><v^>^v^^v<><^>^>><^<<^>>>vv<v<^>v>vv^><v<>v>>^^>^>vv>vv<<^><^vv^>v<<>^^<<><>^v>vv^v^><<^<<<v^>^^<>>v>^^<><^<^<>^<v>>v><>>><>>v><v>>v>^v<<>v^<>^vvv<vv><><<<^<<^<>v^><
"""

In [5]:
solve_part_1(map_str, moves_str)

1487337

In [6]:
solve_part_2(map_str, moves_str)

1521952